In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from joblib import dump, load

In [2]:
class DataLoader:
    def __init__(self, csv_file):
        self.csv_file = csv_file
    
    def read_data(self):
        data = pd.read_csv(self.csv_file)
        return data

class ImageProcessor:
    def preprocess_image(self, image):
        image = cv2.equalizeHist(image)
        image = cv2.GaussianBlur(image, (5, 5), 0)
        if image.shape[0] > 144 or image.shape[1] > 256:
            image = cv2.resize(image, (256, 144))
        return image

    def compute_hog(self, image):
        hog_features = hog(image, pixels_per_cell=(9, 9), cells_per_block=(1, 1), block_norm='L2-Hys')
        return hog_features

    def compute_lbp(self, image):
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp_features = local_binary_pattern(image_gray, P=8, R=3.0, method='uniform')
        return lbp_features.flatten()

class SVMTrainer:
    def __init__(self):
        self.model = SVC(kernel='linear', C=1.0, random_state=42)

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def evaluate(self, X_test, y_test):
        y_pred = self.model.predict(X_test)
        report = classification_report(y_test, y_pred)
        return report

    def save_model(self, model_file):
        dump(self.model, model_file)

    def load_model(self, model_file):
        self.model = load(model_file)

class HandDetector:
    def __init__(self, csv_file):
        self.loader = DataLoader(csv_file)
        self.processor = ImageProcessor()
        self.trainer = SVMTrainer()

    def initialize_dataset(self):
        data = self.loader.read_data()
        X, y = [], []
        for index, row in data.iterrows():
            image_path = row['image_path']
            if not os.path.exists(image_path):
                print(f"File not found: {image_path}")
                continue
            image = cv2.imread(image_path)
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image_processed = self.processor.preprocess_image(image_gray)
            if image_processed is None:
                print(f"Error processing image: {image_path}")
                continue
            num_hands = row['num_hands']
            if num_hands == 0:
                y.append([0, 0, 0, 0, 0, 0, 0, 0])
            elif num_hands == 1:
                x1_min, y1_min = row['1_x_1'], row['1_y_1']
                x1_max, y1_max = row['1_x_2'], row['1_y_2']
                y.append([x1_min, y1_min, x1_max, y1_max, 0, 0, 0, 0])
            elif num_hands == 2:
                x1_min, y1_min = row['1_x_1'], row['1_y_1']
                x1_max, y1_max = row['1_x_2'], row['1_y_2']
                x2_min, y2_min = row['2_x_1'], row['2_y_1']
                x2_max, y2_max = row['2_x_2'], row['2_y_2']
                y.append([x1_min, y1_min, x1_max, y1_max, x2_min, y2_min, x2_max, y2_max])
            hog_features = self.processor.compute_hog(image_processed)
            lbp_features = self.processor.compute_lbp(image_processed)
            X.append(list(hog_features) + list(lbp_features))
        return X, np.array(y)

    def train_model(self, X_train, y_train):
        self.trainer.train(X_train, y_train)

    def evaluate_model(self, X_test, y_test):
        report = self.trainer.evaluate(X_test, y_test)
        return report

    def save_model(self, model_file):
        self.trainer.save_model(model_file)

    def load_model(self, model_file):
        self.trainer.load_model(model_file)

    def predict_and_visualize(self, image_path):
        image = cv2.imread(image_path)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_processed = self.processor.preprocess_image(image_gray)
        hog_features = self.processor.compute_hog(image_processed)
        lbp_features = self.processor.compute_lbp(image_processed)
        features = np.hstack((hog_features, lbp_features))
        predicted_positions = self.trainer.model.predict([features])[0]
        predicted_positions = np.reshape(predicted_positions, (-1, 4))
        for pos in predicted_positions:
            x_min, y_min, x_max, y_max = pos.astype(int)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.imshow('Result', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def predict_and_return_coordinates(self, image_path):
        image = cv2.imread(image_path)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_processed = self.processor.preprocess_image(image_gray)
        hog_features = self.processor.compute_hog(image_processed)
        lbp_features = self.processor.compute_lbp(image_processed)
        features = np.hstack((hog_features, lbp_features))
        predicted_positions = self.trainer.model.predict([features])[0]
        predicted_positions = np.reshape(predicted_positions, (-1, 4))
        return predicted_positions

    def process_camera(self):
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_FPS, 30)
        cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Không thể đọc từ camera")
                break
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_processed = self.processor.preprocess_image(frame_gray)
            hog_features = self.processor.compute_hog(frame_processed)
            lbp_features = self.processor.compute_lbp(frame_processed)
            features = np.hstack((hog_features, lbp_features))
            predicted_positions = self.trainer.model.predict([features])[0]
            predicted_positions = np.reshape(predicted_positions, (-1, 4))
            for pos in predicted_positions:
                x_min, y_min, x_max, y_max = pos
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.imshow('Camera', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [3]:
# Example usage:
if __name__ == "__main__":
    csv_file = 'data/info.csv'
    hand_detector = HandDetector(csv_file)

    # Initialize dataset
    X, y = hand_detector.initialize_dataset()

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    hand_detector.train_model(X_train, y_train)

    # Evaluate model
    report = hand_detector.evaluate_model(X_test, y_test)
    print("Classification report:\n", report)

    # Save model
    model_file = 'models/hand_detection_model.pkl'
    hand_detector.save_model(model_file)

    # Load model (optional)
    hand_detector.load_model(model_file)

    # Predict and visualize from image
    image_path = 'test.jpg'
    hand_detector.predict_and_visualize(image_path)

    # Process camera frames
    hand_detector.process_camera()

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x16764f4e::Set<1,-1,-1>,struct cv::impl::A0x16764f4e::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
